## **파이썬 머신러닝 완벽 가이드 CH9. 8 - Surprise**

0523 월요일 과제 [개념+필사]

### **08 파이썬 추천 시스템 패키지 - Surprise**

##### **Surprise 패키지 소개**

In [1]:
! pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 11.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633734 sha256=a3a8fa0f94809628ad1424ccd6e1862a0040d1afa4c74e172300ea3596f9f3d8
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


##### **Surprise를 이용한 추천 시스템 구축**

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [7]:
data = Dataset.load_builtin('ml-100k', prompt=False) 
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


잠재 요인 협업 필터링 수행

In [8]:
algo = SVD()
algo.fit(trainset)

In [9]:
predictions = algo.test(testset)
print('prediction type:', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.652263161308729, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.650603925606608, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.191770265646958, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.824571660708302, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.64236155542614, details={'was_impossible': False})]

In [10]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.652263161308729),
 ('882', '291', 3.650603925606608),
 ('535', '507', 4.191770265646958)]

In [11]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함.
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.24   {'was_impossible': False}


In [12]:
accuracy.rmse(predictions)

RMSE: 0.9457


0.9457471308768456

##### **Surprise 주요 모듈 소개**

**Dataset**

* Surprise는 user_id, item_id, rating 데이터가 로우 레벨로 된 데이터 세트만 적용
* 칼럼 순서대로 적용하고 네 번째 칼럼부터는 로딩 수행 X


```Dataset.load_builtin(name='ml-100k')``` : 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터 제공. 입력 파라미터인 name으로 대상 데이터가 ml-100k인지 ml-1m인지 입력하는데, 디폴트는 ml-100k


```Dataset.load_from_file(file_path, reader)``` : 콤마, 탭 등으로 칼럼이 분리된 포맷의 OS 파일에서 데이터 로딩. 입력 파라미터로 OS 파일명, Reader로 파일의 포맷을 지정.


```Dataset.load_from_df``` : 판다스의 DataFrame에서 데이터 로딩. 파라미터로 df를 입력받으며 df 역시 반드시 3개의 칼럼인 사용자 아이디, 아이템 아이디, 평점 순으로 칼럼 순서가 정해져 있어야 함. 입력 파라미터로 df 객체, reader로 파일의 포맷을 지정







**OS 파일 데이터를 Surprise 데이터 세트로 로딩**

In [14]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('ratings_noh.csv', index=False, header=False)

In [16]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data = Dataset.load_from_file('ratings_noh.csv', reader=reader)

* 

```line_format(string) ``` : 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식.
* ```sep(char) ``` : 칼럼을 분리하는 분리자, 디폴트는 '\t'. 판다스 df에서 입력받을 때 기재할 필요 X
* ```rating_scale(tuple, optional) ``` : 평점 값의 최소~최대 평점 설정. 디폴트는 (1,5)



In [17]:
trainsest, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)
# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.1641


1.1641435876571864

**판다스 DataFrame에서 Surprise 데이터 세트로 로딩**

In [19]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

# ratings Dataframe에서 칼럼은 사용자 아이디, 아이템 아읻, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8908


0.8907754769926038

##### **Surprise 추천 알고리즘 클래스**

* SVD : 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
* KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
* BaselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

SVD 클래스의 입력 파라미터
* ```n_factors``` : 잠재 요인 K의 개수. 디폴트는 100, 커질수록 정확도가 높아지나 과적합 문제 발생 우려
* ```n_epochs``` : SGD(Stochastic Gradient Descent) 수행 시 반복 횟수. 디폴트는 20
* ```biased(bool)``` : 베이스라인 사용자 편향 적용 여부. 디폴트는 True


##### **베이스라인 평점**

개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 것.

전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수
* 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
* 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
* 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

##### **교차 검증과 하이퍼 파라미터 튜닝**

In [20]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5,5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8943  0.8914  0.9002  0.8930  0.8951  0.8948  0.0030  
MAE (testset)     0.6860  0.6893  0.6939  0.6865  0.6917  0.6895  0.0030  
Fit time          6.48    5.33    5.19    5.32    5.23    5.51    0.49    
Test time         0.22    0.33    0.13    0.37    0.20    0.25    0.09    


{'fit_time': (6.4820709228515625,
  5.327112197875977,
  5.194437503814697,
  5.3185646533966064,
  5.229727029800415),
 'test_mae': array([0.68598308, 0.6892669 , 0.69392382, 0.68651199, 0.69174334]),
 'test_rmse': array([0.89433179, 0.89143542, 0.90018214, 0.89300028, 0.89508364]),
 'test_time': (0.21575021743774414,
  0.33004307746887207,
  0.1331770420074463,
  0.3708949089050293,
  0.2047584056854248)}

In [21]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs':[20,40,60], 'n_factors':[50,100,200]}
# CV를 3개 폴드로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.9007142904385051
{'n_epochs': 20, 'n_factors': 50}


##### **Surprise를 이용한 개인화 영화 추천 시스템 구축**

In [22]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발생합니다.
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: ignored

In [25]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성.
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv',reader=reader)

# 전체 데이터를 학습 데이터로 생성함.
trainset = data_folds.build_full_trainset()

In [26]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [32]:
#영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('movies.csv')

#userId=9의 movieId 데이터 추출해 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [33]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid,iid,verbose=True)

user: 9          item: 42         r_ui = None   est = 2.64   {'was_impossible': False}


In [36]:
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

    # 모든 영화의 movieId를 리스트로 생성.
    total_movies = movies['movieId'].tolist()

    # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies),
          '전체 영화 수:', len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 45 추천 대상 영화 수: 9697 전체 영화 수: 9742


In [38]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):

    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]

    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est

    # sortkey_est() 반환값의 내림차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수: 45 추천 대상 영화 수: 9697 전체 영화 수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.400579409001147
Godfather, The (1972) : 4.370540045092637
It Happened One Night (1934) : 4.363849637541674
Casablanca (1942) : 4.336024874543918
Maltese Falcon, The (1941) : 4.321015160843752
All About Eve (1950) : 4.309431147405599
It's a Wonderful Life (1946) : 4.303661756312046
African Queen, The (1951) : 4.289780175135358
Cinema Paradiso (Nuovo cinema Paradiso) (1989) : 4.283687861376393
Modern Times (1936) : 4.271531364943996
